The contents of this notebook are redistributed and modified from [TorchIO tutorials](https://github.com/fepegar/torchio/tree/main/tutorials) (Transforms) as per the [Apache-2.0 license](https://github.com/fepegar/torchio/blob/main/LICENSE).

# Data preprocessing and augmentation using TorchIO: a tutorial


Preprocessing and augmentation are always overlooked in papers, but these are extremely important and therefore the software and parameters used should be correctly reported. Most importantly, papers will be more easily reproducible if researchers can share the tools they use for these tasks.

![One does not simply throw data into a U-Net](https://i.imgflip.com/4b1gac.jpg)

Transforms are callable Python objects that take data and modify it. In the context of deep learning for medical images, they can be used to pre- or post-process the images, or for data augmentation.

[TorchIO](https://torchio.readthedocs.io/index.html) transforms support 4D tensors (and therefore also 2D and 3D), so it can be used to process most types of medical images: X-rays, CT, 4D ultrasound, structural MRI, diffusion MRI, functional MRI, PET, SPECT, multispectral, RGB, histology...

In this tutorial, we will go through most of the transforms and show some usage examples, including a typical composition of transforms for training and testing.  
Some important concepts of medical imaging are also explained, so it will be helpful for students, researchers or clinicians who are getting started in the field.

If you are using Google Colab, you might find useful to navigate through the tutorial using the table of contents at the left of the screen.

<a href="https://colab.research.google.com/github/AFRICAI-MICCAI/model_development_1_data/blob/main/Notebooks/4-5-%20Data-preprocessing-and-augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Setup

### Conda environment

It is suggested to create a conda environment for the summer school's notebooks. Please find conda installation instructions [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html) (miniconda would be enough).  
If you have not created/initialized the africai conda environment, run in a terminal from the parent  directory *model_development_1_data*:  
> conda env create -f africai.yml  
> conda activate africai

*Other useful commands*:  
To deactivate a conda environment 
> conda deactivate

To delete a conda environment (e.g. africai conda environment, replace *ENV_NAME* with *africai*)
> conda remove --name *ENV_NAME* --all 

### Install PyTorch  before installing TorchIO. It is recommended to use light-the-torch.  
In a terminal, run:
> pip install light-the-torch  
> ltt install torch

Then [install TorchIO](https://torchio.readthedocs.io/quickstart.html#installation) and other libraries, download a couple of files and import the necessary modules.

In [ ]:
%%capture

!pip install torchio==0.18.90 --quiet
!pip install pandas --quiet
!pip install matplotlib --quiet
!pip install seaborn --quiet
!pip install scikit-image --quiet

In [ ]:
import copy
import time
import pprint
import os
from pathlib import Path
from zipfile import ZipFile

import torch
import torchio as tio
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

sns.set_style("whitegrid", {'axes.grid' : False})
%config InlineBackend.figure_format = 'retina'
torch.manual_seed(14041931)

print('TorchIO version:', tio.__version__)
print('Last run on', time.ctime())

data_dir = os.path.join(os.path.dirname(os.getcwd()), "data")
Path(data_dir).mkdir(parents=True, exist_ok=True)

!cd {data_dir} && curl -s -o colormap.txt https://raw.githubusercontent.com/thenineteen/Semiology-Visualisation-Tool/master/slicer/Resources/Color/BrainAnatomyLabelsV3_0.txt
!cd {data_dir} && curl -s -o slice_7t.jpg https://www.statnews.com/wp-content/uploads/2019/08/x961_unsmoothed_cropped-copy-768x553.jpg
!cd {data_dir} && curl -s -o slice_histo.jpg https://upload.wikimedia.org/wikipedia/commons/6/64/Medulloepithelioma_Histology.jpg
!cd {data_dir} && curl -s -o vhp.zip https://data.lhncbc.nlm.nih.gov/public/Visible-Human/Sample-Data/Six%20slices%20from%20the%20Visible%20Male.zip

file_name = os.path.join(data_dir, "vhp.zip")
with ZipFile(file_name, "r") as zip:
    zip.extractall(os.path.join(data_dir, "vhp"))
    print("Input data file unzipped!")

Below are some visualization functions. The cell is hidden by default on Google Colab because the code is not directly related to this tutorial, but you can expand it by double-clicking on it, if you are curious.

In [ ]:
def get_bounds(self):
    """Get image bounds in mm.

    Returns:
        np.ndarray: [description]
    """
    first_index = 3 * (-0.5,)
    last_index = np.array(self.spatial_shape) - 0.5
    first_point = nib.affines.apply_affine(self.affine, first_index)
    last_point = nib.affines.apply_affine(self.affine, last_index)
    array = np.array((first_point, last_point))
    bounds_x, bounds_y, bounds_z = array.T.tolist()
    return bounds_x, bounds_y, bounds_z

def to_pil(image):
    from PIL import Image
    from IPython.display import display
    data = image.numpy().squeeze().T
    data = data.astype(np.uint8)
    image = Image.fromarray(data)
    w, h = image.size
    display(image)
    print()  # in case multiple images are being displayed

def stretch(img):
    p1, p99 = np.percentile(img, (1, 99))
    from skimage import exposure
    img_rescale = exposure.rescale_intensity(img, in_range=(p1, p99))
    return img_rescale

def show_fpg(
        subject,
        to_ras=False,
        stretch_slices=True,
        indices=None,
        intensity_name='t1',
        parcellation=True,
        ):
    subject = tio.ToCanonical()(subject) if to_ras else subject
    def flip(x):
        return np.rot90(x)
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    if indices is None:
        half_shape = torch.Tensor(subject.spatial_shape) // 2
        i, j, k = half_shape.long()
        i -= 5  # use a better slice
    else:
        i, j, k = indices
    bounds_x, bounds_y, bounds_z = get_bounds(subject.t1)  ###

    orientation = ''.join(subject.t1.orientation)
    if orientation != 'RAS':
        import warnings
        warnings.warn(f'Image orientation should be RAS+, not {orientation}+')
    
    kwargs = dict(cmap='gray', interpolation='none')
    data = subject[intensity_name].data
    slices = data[0, i], data[0, :, j], data[0, ..., k]
    if stretch_slices:
        slices = [stretch(s.numpy()) for s in slices]
    sag, cor, axi = slices
    
    axes[0, 0].imshow(flip(sag), extent=bounds_y + bounds_z, **kwargs)
    axes[0, 1].imshow(flip(cor), extent=bounds_x + bounds_z, **kwargs)
    axes[0, 2].imshow(flip(axi), extent=bounds_x + bounds_y, **kwargs)

    kwargs = dict(interpolation='none')
    data = subject.seg.data
    slices = data[0, i], data[0, :, j], data[0, ..., k]
    if parcellation:
        sag, cor, axi = [color_table.colorize(s.long()) if s.max() > 1 else s for s in slices]
    else:
        sag, cor, axi = slices
    axes[1, 0].imshow(flip(sag), extent=bounds_y + bounds_z, **kwargs)
    axes[1, 1].imshow(flip(cor), extent=bounds_x + bounds_z, **kwargs)
    axes[1, 2].imshow(flip(axi), extent=bounds_x + bounds_y, **kwargs)
    
    plt.tight_layout()


class ColorTable:
    def __init__(self, colors_path):
        self.df = self.read_color_table(colors_path)

    @staticmethod
    def read_color_table(colors_path):
        df = pd.read_csv(
            colors_path,
            sep=' ',
            header=None,
            names=['Label', 'Name', 'R', 'G', 'B', 'A'],
            index_col='Label',
        )
        return df

    def get_color(self, label: int):
        """
        There must be nicer ways of doing this
        """
        try:
            rgb = (
                self.df.loc[label].R,
                self.df.loc[label].G,
                self.df.loc[label].B,
            )
        except KeyError:
            rgb = 0, 0, 0
        return rgb

    def colorize(self, label_map: np.ndarray) -> np.ndarray:
        rgb = np.stack(3 * [label_map], axis=-1)
        for label in np.unique(label_map):
            mask = label_map == label
            color = self.get_color(label)
            rgb[mask] = color
        return rgb

color_table = ColorTable(os.path.join(data_dir, 'colormap.txt'))

## Input types

In TorchIO, [transforms](https://torchio.readthedocs.io/transforms/transforms.html) input can be multiple things:
1. TorchIO `Subject`
2. TorchIO `Image`
3. 4D NumPy array
4. 4D PyTorch tensor
5. SimpleITK image
6. Python dictionary

The output type will match the input type.

Let's try first a tiny 4D tensor.

In [ ]:
add_noise = tio.RandomNoise()
tensor = torch.ones(1, 2, 2, 2)
transformed = add_noise(tensor)

print('Before transform:')
print(tensor)
print()
print('After transform:')
print(transformed)

We can create a [`torchio.ScalarImage`]() from it, and the result will be of the same type.

In [ ]:
image = tio.ScalarImage(tensor=tensor)
transformed = add_noise(image)
print(transformed)

Let's try with [`torchio.Subject`]().

In [ ]:
subject = tio.Subject(tiny=image)
transformed = add_noise(subject)
print(transformed)

Python dictionary (mostly for compatibility with [Eisen](https://eisen.ai/) and [MONAI](https://monai.io/)). For this, we need to specify the keys when instantiating the transform:

In [ ]:
python_dict = {'tiny': tensor}
add_noise_dict = tio.RandomNoise(keys=['tiny'])
transformed = add_noise_dict(python_dict)
print(transformed)

And finally, a SimpleITK image:

In [ ]:
sitk_image = image.as_sitk()
transformed = add_noise(sitk_image)
print(type(transformed))

## Spatial transforms

Spatial transforms modify the image bounds or the voxel positions. They are applied to all images simultaneously, as we will se below.

We will use for the tutorial a dataset that can be downloaded with [`torchio.datasets`](https://torchio.readthedocs.io/datasets.html). It contains a 3T structural MRI, a corresponding [brain parcellation](http://niftyweb.cs.ucl.ac.uk/program.php?p=GIF) and two transforms to the [MNI space](https://www.lead-dbs.org/about-the-mni-spaces/), one rigid (6 degrees of freedom (DOF): 3 for rotation and 3 for translation) and one affine (12 DOF: 3 for rotation, 3 for translation, 3 for scaling and 3 for shearing).

Check out [NiBabel docs](https://nipy.org/nibabel/coordinate_systems.html) for an excellent explanation of affine matrices and coordinate systems in medical imaging.

In [ ]:
fpg = tio.datasets.FPG()
print('Sample subject:', fpg)
show_fpg(fpg)

The aspect ratio is distorted and it looks upside down. This is because our visualization function expects images in [`RAS+` orientation](https://nipy.org/nibabel/image_orientation.html), the "canonical" orientation for the NIfTI format, but this image is in `PIR+`. We can use a preprocessing transform to fix that.

### Preprocessing

#### To canonical

Medical images can be [oriented](https://www.slicer.org/wiki/Coordinate_systems) in many different ways. To ensure a consistent orientation across our dataset, we can use a preprocessing transform.

[`ToCanonical`](https://torchio.readthedocs.io/transforms/preprocessing.html#tocanonical) reorganizes the voxels positions and the associated [affine matrix](https://nipy.org/nibabel/coordinate_systems.html#the-affine-matrix-as-a-transformation-between-spaces) so that the image is now in `RAS+` orientation, i.e. the voxels indices will grow from left to right, from posterior (back) to anterior (nose) and from inferior (feet) to superior (head), respectively.

In [ ]:
to_ras = tio.ToCanonical()
fpg_ras = to_ras(fpg)
print('Old orientation:', fpg.t1.orientation)
print('New orientation:', fpg_ras.t1.orientation)
show_fpg(fpg_ras)

As you can see, the same transform is applied to the scalar image and to the label map.

In [ ]:
print(fpg_ras.t1)
print(fpg_ras.seg)

#### Resample

Images are discrete versions of continuous signals, therefore a spatial sampling frequency, and therefore also a period, are always associated with them. The spatial period is the distance between the voxel centers, or spacing. It is also the voxel size. We can check the image spacing (in mm) using the `spacing` property, or printing an image that has been loaded.

In [ ]:
print(fpg_ras.t1)

We might prefer to resize our images to reduce computational burden. For that, we need to *downsample* our image. We can down- or up-sample our images with the [`Resample` transform](https://torchio.readthedocs.io/transforms/preprocessing.html#resample).

We will use an image with 3 times fewer voxels along each dimension than our original version. To keep the image bounds constant, the image spacing needs to be 3 times larger than the original one, i.e., 3 mm. Our final image will contain around $3^3$ times fewer voxels than the full-resolution one!


In [ ]:
downsampling_factor = 3
original_spacing = 1
target_spacing = downsampling_factor / original_spacing  # in mm
downsample = tio.Resample(target_spacing)
downsampled = downsample(fpg_ras)
print('Original image:', fpg_ras.t1)
print('Downsampled image:', downsampled.t1)
print(f'The downsampled image takes {fpg_ras.t1.memory / downsampled.t1.memory:.1f} times less memory!')
show_fpg(downsampled)

The downsampling generates some [aliasing](https://en.wikipedia.org/wiki/Aliasing). We can use concepts of digital signal processing to compute an [optimal](https://link.springer.com/chapter/10.1007/978-3-319-24571-3_81) Gaussian kernel that we will convolve with the image to remove high frequencies before downsampling.

In [ ]:
original_spacing = 1
std = tio.Resample.get_sigma(downsampling_factor, original_spacing)
antialiasing = tio.Blur(std)  # we need don't need a random transform here
blurry = antialiasing(fpg_ras)
downsampled_antialiasing = downsample(blurry)
show_fpg(downsampled_antialiasing)

This looks a bit better than the downsampled image without antialiasing.

Another handy use for the `Resample` transform is to apply a precomputed transformation to a standard space, such as the [MNI space](https://www.lead-dbs.org/about-the-mni-spaces/). The `FPG` dataset includes this transform:

In [ ]:
np.set_printoptions(precision=2, suppress=True)
fpg.t1['affine_matrix'].numpy()

We can provide a reference image for the resampling. Let's download [a dataset in the MNI space](http://www.bic.mni.mcgill.ca/ServicesAtlases/Colin27) included in TorchIO and use it as reference space.

In [ ]:
mni = tio.datasets.Colin27()
to_mni = tio.Resample(mni.t1.path, pre_affine_name='affine_matrix')
fpg_mni = to_mni(fpg_ras)
show_fpg(fpg_mni)

That is our FPG dataset in the MNI space!

#### Crop or pad

Sometimes there is a large part of the image without much information. To reduce the computational burden, we will use [`CropOrPad`](https://torchio.readthedocs.io/transforms/preprocessing.html#croporpad) to crop the image around the brain. If the target shape is larger than the input shape for any dimension, the image will be padded instead along that dimension. We will use a (not helpful) large size along the lateral dimension to show this feature.

In [ ]:
fpg_ras.t1.spatial_shape

In [ ]:
target_shape = 300, 200, 170
crop_pad = tio.CropOrPad(target_shape)
show_fpg(crop_pad(fpg_ras))

If we don't know where the region of interest (ROI) is in the image, but have a segmentation of it, we can use it to perform the cropping/padding by setting the center of the image to be the center of the ROI.

In [ ]:
target_shape = 300, 200, 170
crop_pad = tio.CropOrPad(target_shape, mask_name='seg')
show_fpg(crop_pad(fpg_ras))

If our images are in a standard space, we probably have a good idea of the shape we need so that our image is large enough to fit our ROI but small enough for computations to be less expensive.

In [ ]:
target_shape = 180, 220, 170
crop_pad = tio.CropOrPad(target_shape)
show_fpg(crop_pad(fpg_mni))

### Augmentation

#### Random downsample

Many medical images are acquired with anisotropic spacing, i.e. voxels are not cubes. Researchers typically use anisotropic resampling for preprocessing before feeding the images into a neural network. We can simulate this effect downsampling our image along a specific dimension and resampling back to an isotropic spacing.

To make the transform reproducible, we have passed a `seed` kwarg to the transform.

In [ ]:
random_anisotropy = tio.RandomAnisotropy()
fpg_anisotropic = random_anisotropy(fpg_mni)
print('Applied transforms:')
pprint.pprint(fpg_anisotropic.history)
show_fpg(fpg_anisotropic)

Let's restore our image to its original resolution. Of course, this is a lossy operation, but that's the point! We want to increase the diversity of our dataset so that our models generalize better. Images in the real world have different artifacts and are not always isotropic, so this is a great transform for medical images.

In [ ]:
resample_back = tio.Resample(fpg_mni.spacing)
fpg_restored = resample_back(fpg_anisotropic)
show_fpg(fpg_restored)

#### Random affine

To simulate different positions and size of the patient within the scanner, we can use a [`RandomAffine`](https://torchio.readthedocs.io/transforms/augmentation.html#randomaffine) transform.

To improve visualization, we will use a 2D image and add a grid to it.

In [ ]:
image = tio.ScalarImage(os.path.join(data_dir, 'slice_7t.jpg'))
spacing = image.spacing[0]
print('Downloaded slice:', image)
image.as_pil()

In [ ]:
slice_grid = copy.deepcopy(image)
data = slice_grid.data
white = data.max()
N = 25
data[..., ::N, :, :] = white
data[..., :, ::N, :] = white
slice_grid.as_pil()

In [ ]:
random_affine = tio.RandomAffine()
slice_affine = random_affine(slice_grid)
slice_affine.as_pil()

This transform can be used to zoom in or out, changing the `scales` kwarg.

In [ ]:
random_affine_zoom = tio.RandomAffine(scales=(2, 2))
slice_affine = random_affine_zoom(slice_grid)
slice_affine.as_pil()

#### Random flip

Flipping images is a very cheap way to perform data augmentation. In medical images, it's very common to flip the images horizontally. We can specify the dimensions indices when instantiating a [`RandomFlip`](https://torchio.readthedocs.io/transforms/augmentation.html#torchio.transforms.RandomFlip) transform.

However, if we don't know the image orientation, we can't know which dimension corresponds to the lateral axis. In TorchIO, you can use anatomical labels instead, so that you don't need to figure out image orientation to know which axis you would like to flip.

To make sure the transform modifies the image, we will use the inferior-superior (longitudinal) axis and a flip probability of 1. If the flipping happened along any other axis, we might not notice it using this visualization.

In [ ]:
random_flip = tio.RandomFlip(axes=['inferior-superior'], flip_probability=1)
fpg_flipped = random_flip(fpg_ras)
show_fpg(fpg_flipped)

#### Random elastic deformation

To simulate anatomical variations in our images, we can apply a non-linear deformation using [`RandomElasticDeformation`](https://torchio.readthedocs.io/transforms/augmentation.html#torchio.transforms.RandomElasticDeformation).

In [ ]:
max_displacement = 15, 10, 0  # in x, y and z directions
random_elastic = tio.RandomElasticDeformation(max_displacement=max_displacement)
slice_elastic = random_elastic(slice_grid)
slice_elastic.as_pil()

 As explained in the [documentation](https://torchio.readthedocs.io/transforms/augmentation.html#torchio.transforms.RandomElasticDeformation), one can change the number of grid control points to set the deformation smoothness.

In [ ]:
random_elastic = tio.RandomElasticDeformation(
    max_displacement=max_displacement,
    num_control_points=20,
)
slice_large_displacement_more_cp = random_elastic(slice_grid)
slice_large_displacement_more_cp.as_pil()

As you can see, TorchIO has raised a warning explaining that our displacement is too large. If this happens, try using a smaller maximum displacement or fewer control points.

Let's look at the effect of using few control points but very large displacements.

In [ ]:
random_elastic = tio.RandomElasticDeformation(
    max_displacement=2 * np.array(max_displacement),
)
slice_large_displacement = random_elastic(slice_grid)
slice_large_displacement.as_pil()

## Intensity transforms

Intensity transforms modify only scalar images, whereas label maps are left as they were.

### Preprocessing (normalization)

#### Rescale intensity

We can change the intensities range of our images so that it lies within e.g. 0 and 1, or -1 and 1, using [`RescaleIntensity`](https://torchio.readthedocs.io/transforms/preprocessing.html#rescaleintensity).

In [ ]:
rescale = tio.RescaleIntensity((-1, 1))
rescaled = rescale(fpg)
fig, axes = plt.subplots(2, 1)
sns.distplot(fpg.t1.data, ax=axes[0], kde=False)
sns.distplot(rescaled.t1.data, ax=axes[1], kde=False)
axes[0].set_title('Original histogram')
axes[1].set_title('Intensity rescaling')
axes[0].set_ylim((0, 1e6))
axes[1].set_ylim((0, 1e6))
plt.tight_layout()

There seem to be some outliers with very high intensity. We might be able to get rid of those by mapping some percentiles to our final values.

In [ ]:
rescaled = rescale(fpg_ras)
fig, axes = plt.subplots(2, 1)
sns.distplot(fpg.t1.data, ax=axes[0], kde=False)
sns.distplot(rescaled.t1.data, ax=axes[1], kde=False)
axes[0].set_title('Original histogram')
axes[1].set_title('Intensity rescaling with percentiles 1 and 99')
axes[0].set_ylim((0, 1e6))
axes[1].set_ylim((0, 1e6))
plt.tight_layout()
show_fpg(rescaled)

#### Z-normalization

Another common approach for normalization is forcing data points to have zero-mean and unit variance. We can use [`ZNormalization`](https://torchio.readthedocs.io/transforms/preprocessing.html#znormalization) for this.

In [ ]:
standardize = tio.ZNormalization()
standardized = standardize(fpg)
fig, axes = plt.subplots(2, 1)
sns.distplot(fpg.t1.data, ax=axes[0], kde=False)
sns.distplot(standardized.t1.data, ax=axes[1], kde=False)
axes[0].set_title('Original histogram')
axes[1].set_title('Z-normalization')
axes[0].set_ylim((0, 1e6))
axes[1].set_ylim((0, 1e6))
plt.tight_layout()

The second mode in our distribution, corresponding to the foreground, is far from zero because the background contributes a lot to the mean computation. We can compute the stats using e.g. values above the mean only. Let's see if the mean is a good threshold to segment the foreground.

In [ ]:
fpg_thresholded = copy.deepcopy(fpg_ras)
data = fpg_thresholded.t1.data
data[data > data.float().mean()] = data.max()
show_fpg(fpg_thresholded)

It seems reasonable to use this mask to compute the stats for our normalization transform.

In [ ]:
standardize_foreground = tio.ZNormalization(masking_method=tio.ZNormalization.mean)
standardized_foreground = standardize_foreground(fpg)
fig, axes = plt.subplots(2, 1)
sns.distplot(fpg.t1.data, ax=axes[0], kde=False)
sns.distplot(standardized_foreground.t1.data, ax=axes[1], kde=False)
axes[0].set_title('Original histogram')
axes[1].set_title('Z-normalization using foreground stats')
axes[0].set_ylim((0, 1e6))
axes[1].set_ylim((0, 1e6))
plt.tight_layout()

The second mode is now closer to zero, as only the foreground voxels have been used to compute the statistics.

#### Histogram standardization

[Histogram standardization](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.204.102&rep=rep1&type=pdf) is yet another technique we can use to normalize our data. First, a set of landmarks is created from a training set. Then, images can be normalized by matching their histograms to the training landmarks.

We will use landmarks derived from the [ADNI](http://adni.loni.usc.edu/) dataset, computed by [Li et al](https://arxiv.org/abs/1707.01992).

In [ ]:
# From NiftyNet model zoo
LI_LANDMARKS = "0 8.06305571158 15.5085721044 18.7007018006 21.5032879029 26.1413278906 29.9862059045 33.8384058795 38.1891334787 40.7217966068 44.0109152758 58.3906435207 100.0"
LI_LANDMARKS = np.array([float(n) for n in LI_LANDMARKS.split()])
landmarks_dict = {'t1': LI_LANDMARKS}
hist_standardize = tio.HistogramStandardization(landmarks_dict, masking_method=tio.ZNormalization.mean)
hist_standard = hist_standardize(fpg_ras)
fig, axes = plt.subplots(2, 1)
sns.distplot(fpg.t1.data, ax=axes[0], kde=False)
sns.distplot(hist_standard.t1.data, ax=axes[1], kde=False)
axes[0].set_title('Original histogram')
axes[1].set_title('Histogram standardization using foreground stats')
axes[0].set_ylim((0, 1e6))
axes[1].set_ylim((0, 1e6))
plt.tight_layout()
show_fpg(hist_standard)

### Augmentation

#### Random blur

We can use [`RandomBlur`](https://torchio.readthedocs.io/transforms/augmentation.html#torchio.transforms.RandomBlur) to smooth/blur the images. The standard deviations of the Gaussian kernels are expressed in mm and will be computed independently for each axis.

In [ ]:
blur = tio.RandomBlur()
blurred = blur(fpg_ras)
show_fpg(blurred)

#### Random noise

Gaussian noise can be simulated using [`RandomNoise`](https://torchio.readthedocs.io/transforms/augmentation.html#randomnoise). This transform is easiest to use after [`ZNormalization`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.ZNormalization), as we know beforehand that the mean and standard deviation of the input will be 0 and 1, respectively. If necessary, the noise `mean` and `std` can be set using the corresponding keyword arguments.

Noise in MRI is actually [Rician](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2254141/), but it is nearly Gaussian for SNR > 2 (i.e. foreground).

In [ ]:
add_noise = tio.RandomNoise(std=0.5)
standard = standardize(fpg_ras)
noisy = add_noise(standard)
show_fpg(noisy)

#### MRI-specific transforms

TorchIO includes some transforms to simulate image artifacts specific to MRI modalities.

##### Random bias field

Magnetic field inhomogeneities in the MRI scanner produce low-frequency intensity distortions in the images, which are typically corrected using algorithms such as [N4ITK](https://pubmed.ncbi.nlm.nih.gov/20378467/). To simulate this artifact, we can use [`RandomBiasField`](https://torchio.readthedocs.io/transforms/augmentation.html#torchio.transforms.RandomBiasField).

For this example, we will use an image that has been preprocessed so it's meant to be unbiased.

In [ ]:
add_bias = tio.RandomBiasField(coefficients=1)
mni.seg = mni.brain
show_fpg(mni)
mni_bias = add_bias(mni)
mni_bias.seg = mni_bias.brain
show_fpg(mni_bias)

The artifact strength is proportional to the `coefficients` kwarg. The polynomial order is related to the artifact frequency, and the transform run time is proportional grows with it.

##### $k$-space transforms

MR images are generated by computing the inverse Fourier transform of the $k$-space, which is the signal received by the coils in the scanner. If the $k$-space is altered, an artifact will be created in the image. These artifacts are typically accidental, but we can use transforms to simulate them.

##### Random spike

Sometimes, signal peaks can appear in $k$-space. If one adds a high-energy component at e.g. 440 Hz in the spectrum of an audio signal, a tone of that frequency will be audible in the time domain. Similarly, spikes in $k$-space manifest as stripes in image space. They can be simulated using [`RandomSpike`](https://torchio.readthedocs.io/transforms/augmentation.html#torchio.transforms.RandomSpike). The number of spikes doesn't affect the transform run time, so try adding more!

In [ ]:
add_spike = tio.RandomSpike()
with_spike = add_spike(fpg_ras)
show_fpg(with_spike)

##### Random ghosting

Ghosting artifacts, caused by patient motion, can be simulated by removing every $n$th plane from the k-space, and can be generated using [`RandomGhosting`](https://torchio.readthedocs.io/transforms/augmentation.html#randomghosting). As with the previous transform, the number of ghosts doesn't affect the run time, so you can add as many as you like.

In [ ]:
add_ghosts = tio.RandomGhosting(intensity=1.5)
with_ghosts = add_ghosts(fpg_ras)
show_fpg(with_ghosts)

##### Random motion

If the patient moves during the MRI acquisition, motion artifacts will be present. TorchIO includes an implementation of [Shaw et al.](http://proceedings.mlr.press/v102/shaw19a.html), where the artifact is generated by filling the $k$-space with random rigidly-transformed versions of the original images and computing the inverse transform of the compound $k$-space.

Computing the direct and inverse Fourier transform takes some time, so we'll use nearest neighbor interpolation to resample faster. Another way of cutting down the run time is using a smaller number of transforms (i.e., the patient moves less during acquisition time).

In [ ]:
add_motion = tio.RandomMotion(num_transforms=6, image_interpolation='nearest')
with_motion = add_motion(fpg_ras)
show_fpg(with_motion)

## Lambda

The fastest way to implement a custom transform is using [`Lambda`](https://torchio.readthedocs.io/transforms/others.html#lambda). This transform needs a callable object which applies a transform to a 4D image. We will use a histology slice to show a usage example.

In [ ]:
rgb = tio.ScalarImage(os.path.join(data_dir, 'slice_histo.jpg'))
rgb.as_pil()

We will pass a function that swaps the red and blue channels of the RGB image.

In [ ]:
def rgb_to_bgr(tensor):
    tensor = torch.from_numpy(tensor.numpy()[::-1].copy())
    return tensor

transform = tio.Lambda(rgb_to_bgr)
bgr = transform(rgb)
bgr.as_pil()

## Composition

It's useful to chain transforms together so that they can be applied with a single statement, instead of a `for` loop.


### Compose

Transforms can be easily chained together using [`Compose`](https://torchio.readthedocs.io/transforms/augmentation.html#torchio.transforms.Compose). It is like [`torchvision.Compose`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Compose), but it takes a `p` kwarg representing the probability that the composition will be applied.

In [ ]:
nice_downsample = tio.Compose([
    to_ras,
    antialiasing,
    downsample,                   
])
nice_down = nice_downsample(fpg)
show_fpg(nice_down)

### "One of"

Sometimes we would like to apply e.g. either a `RandomAffine` or a `RandomElasticDeformation`, instead of both. However, computing a `RandomAffine` is faster. Therefore, we might want it to happen to only 25% of images, not 50%. We can use [`OneOf`](https://torchio.readthedocs.io/transforms/augmentation.html#oneof) for this.

In [ ]:
spatial_transform = tio.OneOf({
    tio.RandomAffine(): 0.75,
    tio.RandomElasticDeformation(): 0.25,
})

### Composition example

This an example of a possible composition of transforms used to train a convolutional neural network. Sometimes the order matters, sometimes it doesn't. Think deeply about the order of your transforms! For example, if you add noise with `RandomNoise` and then blur with `RandomBlur`, you'll be removing a lot of that noise!

Do **not** use a seed for your random transforms during training! If you do, the same transform will be applied to every image.

In [ ]:
get_foreground = tio.ZNormalization.mean

training_transform = tio.Compose([
    tio.Resample(
        mni.t1.path,
        pre_affine_name='affine_matrix'),      # to MNI space (which is RAS+)
    tio.RandomAnisotropy(p=0.25),              # make images look anisotropic 25% of times
    tio.CropOrPad((180, 220, 170)),            # tight crop around brain
    tio.HistogramStandardization(
        landmarks_dict,
        masking_method=get_foreground),        # standardize histogram of foreground
    tio.ZNormalization(
        masking_method=get_foreground),        # zero mean, unit variance of foreground
    tio.RandomBlur(p=0.25),                    # blur 25% of times
    tio.RandomNoise(p=0.25),                   # Gaussian noise 25% of times
    tio.OneOf({                                # either
        tio.RandomAffine(): 0.8,               # random affine
        tio.RandomElasticDeformation(): 0.2,   # or random elastic deformation
    }, p=0.8),                                 # applied to 80% of images
    tio.RandomBiasField(p=0.3),                # magnetic field inhomogeneity 30% of times
    tio.OneOf({                                # either
        tio.RandomMotion(): 1,                 # random motion artifact
        tio.RandomSpike(): 2,                  # or spikes
        tio.RandomGhosting(): 2,               # or ghosts
    }, p=0.5),                                 # applied to 50% of images
])

fpg_training = tio.datasets.FPG()
fpg_augmented = training_transform(fpg_training)         # apply the transform
show_fpg(fpg_augmented)

Applying many transforms to large images can be slow. Tips to go faster:
1. Use nearest neighbor interpolation
2. Crop the volumes
3. Downsample the images
4. Use `OneOf` and the `p` kwarg wisely
5. Use a small number of transforms in `RandomMotion`
6. Use multiple workers in a `DataLoader`

Which random transforms have been applied? Using what parameters? All the information is saved in the `history` attribute of [`Subject`](https://torchio.readthedocs.io/data/subject.html).

In [ ]:
pprint.pprint(fpg_augmented.history)

At test time, we only need the preprocessing transforms:

In [ ]:
testing_transform = tio.Compose([
    tio.Resample(mni.t1.path, pre_affine_name='affine_matrix'),                   # to MNI space (which is RAS+)
    tio.HistogramStandardization(landmarks_dict, masking_method=get_foreground),  # standard histogram
    tio.ZNormalization(masking_method=get_foreground),                            # zero mean and unit std
])

fpg_testing = testing_transform(fpg)
show_fpg(fpg_testing)

## Implementations of recent papers

TorchIO transforms can be written to implement more complex or higher-level operations. These are some examples of transforms that implement recent studies or have been used in them.

#### Random swap

[Chen et al.](https://www.sciencedirect.com/science/article/abs/pii/S1361841518304699) randomly swapped small image patches to perform image restoration as a pretext task for self-supervised learning. This can be replicated using [`RandomSwap`](https://torchio.readthedocs.io/transforms/augmentation.html#randomswap). You'll want to use a smaller number of iterations for 2D images.

In [ ]:
swap = tio.RandomSwap()
swapped = swap(fpg_mni)
show_fpg(swapped)

#### Random labels to image

[Billot et al.](https://github.com/BBillot/SynthSeg) generated images with different contrasts using only tissue segmentations, obtaining great results for modality-agnostic image segmentation. [`RandomLabelsToImage`](https://torchio.readthedocs.io/transforms/augmentation.html#randomlabelstoimage) can be used to generate this type of images.

The authors recommend smoothing the image after, for which we can use `RandomBlur` with deterministic parameters.

In [ ]:
print('Downloading dataset...')
large_colin = tio.datasets.Colin27(version=2008)
new_colin = tio.Subject(t1=large_colin.t1, seg=large_colin.cls)  # ignore T2 and PD
print('Loading data...')
new_colin.load()  # load data here so that transform run time is not affected
print('Resampling...')
smaller_colin = tio.Resample(2, image_interpolation='nearest')(new_colin)
print('Applying transform...')
lab2im = tio.RandomLabelsToImage(label_key='seg', image_key='synthetic')
lab2im_blur = tio.RandomBlur((1, 1))
labels_image = lab2im(smaller_colin)
labels_image.synthetic.data[labels_image.seg.data == 0] = 0  
labels_image_smooth = lab2im_blur(labels_image)
show_fpg(labels_image_smooth, intensity_name='synthetic', parcellation=False)

#### Random resection (working better on Colab)

[Pérez-García et al.](https://arxiv.org/abs/2006.15693) simulated brain resection cavities from healthy subjects to pre-train a segmentation model using self-supervised learning. We will first download and install the [`resector`](https://github.com/fepegar/resector) package.

In [ ]:
!pip install --quiet resector

Now we can use the `resect` command to apply to the transform. The first time it is run, it will take some time as it needs to generate some files necessary to simulate the resection.

In [ ]:
fpg_mni.t1.save(os.path.join(data_dir, 't1.nii.gz'))
fpg_mni.seg.save(os.path.join('gif_parcellation.nii.gz'))
!resect t1.nii.gz gif_parcellation.nii.gz t1_resected.nii.gz t1_resection_label.nii.gz --seed 42

In [ ]:
resected = tio.Subject(
    t1=tio.ScalarImage(os.path.join(data_dir, 't1_resected.nii.gz')),
    seg=tio.LabelMap(os.path.join(data_dir, 't1_resection_label.nii.gz'))
)
centroid = np.array(np.where(resected.seg.data)).mean(axis=1).astype(np.uint16)[-3:]
show_fpg(resected, indices=centroid)

## Other interfaces

You can still try out the transforms using a GUI or a CLI tool even without using Python.

### 3D Slicer

[3D Slicer](https://www.slicer.org/) is *an open source software platform for medical image informatics, image processing, and three-dimensional visualization*.

The [extensions index](https://www.slicer.org/wiki/Documentation/Nightly/SlicerApplication/ExtensionsManager) include the [TorchIO](https://torchio.readthedocs.io/slicer.html) extension, that can be used to try out the transforms and their parameters.

![TorchIO extension screenshot](https://raw.githubusercontent.com/fepegar/SlicerTorchIO/master/Screenshots/TorchIO.png)

### Command-line tool

After installing TorchIO with `pip`, you can just run the [`torchio-transform`](https://torchio.readthedocs.io/cli.html) command to apply a transform to a image file.

In [ ]:
fpg_mni.t1.save(os.path.join(data_dir, 't1.nii.gz'))
!tiotr --kwargs "degrees=20 scales=(0.5,2)" --seed 42 ../data/t1.nii.gz RandomAffine ../data/t1_cli.nii.gz

We can also use tiohd to check the header and plot the image, although plotting won't work on Colab.

In [ ]:
!tiohd --plot ../data/t1_cli.nii.gz

## Conclusion

Preprocessing and augmentation of medical images are important but not trivial steps in all deep learning pipelines. In this tutorial, we have seen that [TorchIO transforms](https://torchio.readthedocs.io/transforms/transforms.html) can be used to perform these tasks in a simple, reproducible way, without reinventing the wheel.

If you want to give feedback, as for features, report bugs or contribute, feel free to [open an issue](https://github.com/fepegar/torchio/issues/new/choose) on [GitHub](https://github.com/fepegar/torchio/)!

## Exercise

1. Use a slice from the [Visible Human Project](https://www.nlm.nih.gov/research/visible/visible_human.html) (VHP),instantiated in the cell below
2. Flip it in the `'anteroposterior'` axis (you'll need to set a kwarg so that flipping happens always)
2. Flip it in the `'lateral'` axis with a probability of 0.5
2. Crop the image around the center using a target shape of `(800, 800, 1)`
3. Resample to 0.75 mm isotropic
4. Perform an elastic deformation using a maximum displacement of 50 mm **or** an affine transformation, with a 60% probability that the elastic transformation will be chosen
5. Blur the image with a probability of 0.6
6. Add Gaussian noise with mean $\mu = 128$ and standard deviation $\sigma = 10$ and rescale the image intensity to the $[0, 255]$ range to avoid overflow artifacts when displaying the image **if** Gaussian noise was added
8. Create a transform to convert the RGB tensor to grayscale*
9. Concatenate all these transforms
10. Apply the concatenated transform to the VHP slice 10 times and display the results using `to_pil`

*Although this should be done [carefully](https://en.wikipedia.org/wiki/Grayscale#Colorimetric_(perceptual_luminance-preserving)_conversion_to_grayscale), an easy way is using the mean of the three (RGB) channels. Remember channels are stored in the first dimension in TorchIO. Tip: use `keepdim=True` in `torch.mean`.

**Bonus**

- Apply the transforms one by one, showing each transform name (`transform.name`) and run time. Print also the total run time. Finally, display each transformed image
- Remove the cropping and resampling transforms and measure the times again. Are they different? Why?